This notebook is for the project on Coursera Capstone Week 4

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
print("Hello! This is my Capstone Project week 4")

Hello! This is my Capstone Project week 4


In [3]:
from urllib import request,parse,error
import ssl
import requests
import json

In [4]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

In [5]:
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium

In [6]:
# 25.0375198,121.5636796
col_names=['District','Latitude','Longitude']
taipei_district = pd.read_csv('Taipei.csv',sep=',',header=None,)#,names=col_names)

In [7]:
#taipei_district.drop(axis=1, columns=[3,4],inplace=True)
taipei_district.columns = col_names
taipei_district

,District,Latitude,Longitude
0,Zhongzheng District,25.024056,121.512136
1,Xinyi District,25.032145,121.591015
2,Nangang District,25.058466,121.622729
3,Datong District,25.048086,121.518874
4,Da’an District,25.017134,121.560030
5,Songshan District,25.051313,121.572990
6,Wanhua District,25.013673,121.501150


In [11]:
address = 'Taipei'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Taipei City are {},{}.'.format(latitude, longitude))

The geograpical coordinate of Taipei City are 25.0375198,121.5636796.


In [13]:
#print('Request_Remaining: {}\nRequest_limit: {}'.format(response.headers['X-RateLimit-Remaining'],\
#response.headers['X-RateLimit-Limit']))

In [14]:
def getNearbyVenues(latitudes, longitudes, radius=2000,limit=100):
    venues_list=[]
    
    clientid = "5PM542SKNZ2I5CILMMSP4OIUNFFWHYRYUM3VAV45HEJCMX1V"
    clientsecret = "POQVL03MINSUUFQNO4MA40YXB4HZ5U2YW5SYOEK1S1KZOZON"
    version = '20190601'
    searchitem = 'coffee'
    for latitude,longitude in zip(latitudes,longitudes):
        #.url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(clientid,clientsecret,version,latitude,longitude,radius,limit)
        url="https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}".format(clientid,clientsecret,version,latitude,longitude,searchitem,radius,limit)
        
        response = requests.get(url).json()['response']['venues']    #['groups'][0]['items']                
        venues_list.append([(            
            v['name'],
            v['location']['lat'],
            v['location']['lng'],
            'District',
            ) for v in response])
        #print(venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Venue',
                            'Latitude',
                            'Longitude',
                            'PostalCode',]
    return(nearby_venues)

In [15]:
taipei_venues = getNearbyVenues(latitudes= taipei_district['Latitude'],longitudes = taipei_district['Longitude'])

In [16]:
map_taipei = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, district in zip(taipei_district['Latitude'],taipei_district['Longitude'],taipei_district['District']):
    label='{}'.format(district)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_taipei)

map_taipei

In [17]:
map_taipei = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, venue, category in zip(taipei_venues['Latitude'],taipei_venues['Longitude'],taipei_venues['Venue'],taipei_venues['PostalCode']):
    label='{},{}'.format(category,venue)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_taipei)

map_taipei

In [18]:
taipei_coffee = []
for venue in taipei_venues['Venue']:    
    taipei_coffee.append(venue.split('(')[0].strip())

In [19]:
taipei_venues['Name']= taipei_coffee